In [ ]:
import torch.optim.lr_scheduler as lr_scheduler

# Charge Integration

In [ ]:
def charge_integration(discrete_data,CI_parameters, starting_index = 0):
  sg = int(CI_parameters[0])
  lg = int(CI_parameters[1])
  st_pt = int(CI_parameters[2])
  THR = CI_parameters[3]

  ql = np.sum(discrete_data[:, st_pt:st_pt + lg], axis=1)
  qs = np.sum(discrete_data[:, st_pt:st_pt + sg], axis=1)
  CI_values = (ql - qs) / ql
  CI_labels = np.where(CI_values > THR, 0, 1)

  return CI_labels, CI_values

def optimize_charge_integration_parameters(discrete_data,discrete_labels, starting_index = 0):
  best_accuracy = 0
  best_sg = 0
  best_lg = 0
  best_PSD_THR = 0
  best_ci_st_pt = 0

  for sg in CI_param1_lst:
    for lg in CI_param2_lst:
      for st_pt in CI_param3_lst:
        if sg >= lg:
          break

        ql = np.sum(discrete_data[:, st_pt:st_pt + lg], axis=1)
        qs = np.sum(discrete_data[:, st_pt:st_pt + sg], axis=1)
        psd_ratio = (ql - qs) / ql

        for THR in CI_param4_lst:
          correct = np.sum(np.where(psd_ratio > THR, 0, 1) == discrete_labels)
          accuracy = 100*correct/len(discrete_labels)
          if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_sg = sg
            best_lg = lg
            best_PSD_THR = THR
            best_ci_st_pt = st_pt

  return [best_sg,best_lg,best_ci_st_pt,best_PSD_THR]

# Correlation

In [ ]:
def correlation_classifier(discrete_data,correlation_parameters, starting_index = 0):
  st_pt = int(correlation_parameters[0])
  temp_end = int(correlation_parameters[1])

  curr_temp_n = np.copy(template_n)
  curr_temp_g = np.copy(template_g)
  curr_temp_n = curr_temp_n[st_pt:temp_end]
  curr_temp_g = curr_temp_g[st_pt:temp_end]
  curr_data = discrete_data[:,time_offset + st_pt:time_offset + temp_end]

  mean_n = np.mean(curr_temp_n)
  sigma_n = np.std(curr_temp_n)
  mean_g = np.mean(curr_temp_g)
  sigma_g = np.std(curr_temp_g)
  mean_x = np.mean(curr_data, axis=1)
  sigma_x = np.std(curr_data, axis=1)
  corr_n = np.dot(curr_data - mean_x[:, np.newaxis], curr_temp_n - mean_n) / (temp_end * sigma_x * sigma_n)
  corr_g = np.dot(curr_data - mean_x[:, np.newaxis], curr_temp_g - mean_g) / (temp_end * sigma_x * sigma_g)

  corr_labels = np.where(corr_n > corr_g, 0, 1)
  corr_values = corr_n - corr_g

  return corr_labels, corr_values

def optimize_correlation_parameters(discrete_data,discrete_labels, starting_index = 0):

  best_accuracy = 0
  best_temp_end = 0
  best_st_pt = 0

  for st_pt in corr_param1_lst:
    for temp_end in corr_param2_lst:

      curr_temp_n = np.copy(template_n)
      curr_temp_g = np.copy(template_g)
      curr_temp_n = curr_temp_n[st_pt:temp_end]
      curr_temp_g = curr_temp_g[st_pt:temp_end]
      curr_data = discrete_data[:,time_offset + st_pt: time_offset + temp_end]

      mean_n = np.mean(curr_temp_n)
      sigma_n = np.std(curr_temp_n)
      mean_g = np.mean(curr_temp_g)
      sigma_g = np.std(curr_temp_g)
      mean_x = np.mean(curr_data, axis=1)
      sigma_x = np.std(curr_data, axis=1)

      corr_n = np.dot(curr_data - mean_x[:, np.newaxis], curr_temp_n - mean_n) / (temp_end * sigma_x * sigma_n)
      corr_g = np.dot(curr_data - mean_x[:, np.newaxis], curr_temp_g - mean_g) / (temp_end * sigma_x * sigma_g)

      correct = np.sum(np.where(corr_n > corr_g, 0, 1) == discrete_labels)
      accuracy = 100*correct/len(discrete_labels)
      if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_temp_end = temp_end
        best_st_pt = st_pt

  return [best_st_pt,best_temp_end]

# Fully Connected NN (DNN applied to x_t)

In [ ]:
class Model_FC_NN(nn.Module):
  def __init__(self,dim=WL_classifier):
    super().__init__()
    self.fct0 = nn.Linear(dim,model_classifier_layer1)
    self.fct = nn.Linear(model_classifier_layer1,model_classifier_layer2)
    self.fc1 = nn.Linear(model_classifier_layer2,model_classifier_layer3)
    self.fc2 = nn.Linear(model_classifier_layer3,model_classifier_layer4)
    self.fc3 = nn.Linear(model_classifier_layer4,model_classifier_layer5)
    self.fc4 = nn.Linear(model_classifier_layer5,1)

  def forward(self,x):
    x = self.fct0(x)
    x = F.relu(x)
    x = self.fct(x)
    x = F.relu(x)
    x = self.fc1(x)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    x = self.fc3(x)
    x = F.relu(x)
    x = self.fc4(x)
    x = F.sigmoid(x)

    return x


def train_Model_FC_NN(model, train_data, train_labels, val_data, val_labels, learning_rate, batch_size, epochs, patience=20, factor=0.2):

    flag_start_again = 0
    while(flag_start_again != 2):
      flag_start_again = 0
      optimizer = optim.Adam(model.parameters(), lr=learning_rate)
      criterion = nn.BCELoss()
      scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=patience, factor=factor, verbose=True)

      losses, epoch_lst, train_accs, val_accs = [], [], [], []
      length_train = np.shape(train_data)[0]
      length_val = np.shape(val_data)[0]

      best_val_acc = 0
      tj = 0
      flagy = 0
      for j in range(epochs):
          if flagy == 1 or flag_start_again == 1:
            break
          perm = np.random.permutation(length_train)
          train_data = train_data[perm]
          train_labels = train_labels[perm]

          for i in range(0, length_train, batch_size):
              if (i + batch_size) > length_train:
                  break
              y_in = train_data[i:i+batch_size, :]
              labels_in = train_labels[i:i+batch_size]
              y_in = torch.Tensor(y_in)
              labels_in = torch.Tensor(labels_in).float().unsqueeze(1)

              output = model(y_in)
              loss = criterion(output, labels_in)
              optimizer.zero_grad()
              loss.backward()
              optimizer.step()

              if i % int(batch_size*300) == 0:
                losses.append(float(loss)/batch_size)  # compute *average* loss
                epoch_lst.append(tj)
                train_cost = float(loss.detach().numpy())
                train_acc = estimate_accuracy_Model_FC_NN(model, train_data, train_labels)
                train_accs.append(train_acc)
                val_acc = estimate_accuracy_Model_FC_NN(model, val_data, val_labels)
                val_accs.append(val_acc)
                print("Epoch %d. [Val Acc %.2f%%] [Train Acc %.2f%%, Loss %f]" % (
                      j, val_acc, train_acc, train_cost))
                tj += 1

                scheduler.step(val_acc)
                for param_group in optimizer.param_groups:
                    curr_lr = param_group['lr']
                if curr_lr == learning_rate*np.power(factor,3):
                    print("Training stopped. No improvement in validation accuracy for %d epochs." % patience)
                    flagy = 1
                    flag_start_again = 2
                    break

                if tj == 10 and val_acc < 53:
                  flag_start_again = 1

    del y_in
    del labels_in
    return losses, epoch_lst, train_accs, val_accs

def estimate_accuracy_Model_FC_NN(model, data, labels):

    data = torch.Tensor(data)
    labels = torch.Tensor(labels).float().unsqueeze(1)
    out = model(data)
    correct_mask = torch.round(out) == labels
    correct = correct_mask.sum().item()
    all = correct_mask.numel()

    return 100 * correct / all

def create_labels_Model_FC_NN(model,data,params):

  data = (data - params[0])/params[1]
  data = torch.Tensor(data)
  nn_values = model(data)
  labels_out = torch.round(nn_values)

  return np.squeeze(labels_out.detach().numpy()), np.squeeze(nn_values.detach().numpy())


def run_Model_FC_NN(labels_train,data_train,labels_val,data_val,labels_test,data_test,path1, path2, learning_rate=1e-3, batch_size=50, epochs=100):
  model = Model_FC_NN()

  mu, sigma = find_normal_parameters(data_train)
  norm_train,norm_val,norm_test = normalize_datasets(data_train,data_val,data_test,mu,sigma)

  learning_curve_info = train_Model_FC_NN(model,norm_train
                                                   , labels_train, norm_val
                                                   , labels_val, learning_rate, batch_size, epochs)

  plot_learning_curve(*learning_curve_info)

  test_accuracy = estimate_accuracy_Model_FC_NN(model, norm_test, labels_test)
  print("[Test Acc %.2f%%]" % (test_accuracy))

  torch.save(model, path1)
  np.save(path2,np.array([mu,sigma,WL_classifier,learning_rate,batch_size,epochs]))

  return

# Mode 1: DNN applied to x_t + y_t. Mode 2: Deep Classifier

In [ ]:
class Model_Attention(nn.Module):
  def __init__(self,y_dim = WL_classifier, z_dim = num_classification_features, m_dim = num_attention_features):
    super().__init__()
    self.fc1 = nn.Linear((y_dim+z_dim), model_classifier_layer1)
    self.fc2 = nn.Linear(model_classifier_layer1,model_classifier_layer2)
    self.fc3 = nn.Linear(model_classifier_layer2,model_classifier_layer3)
    self.fc4 = nn.Linear(model_classifier_layer3,model_classifier_layer4)
    self.fc5 = nn.Linear(model_classifier_layer4,model_classifier_layer5)
    self.fc6 = nn.Linear(model_classifier_layer5,1)

    self.r_fc1 = nn.Linear(m_dim, model_attention_layer1)
    self.r_fc2 = nn.Linear(model_attention_layer1, model_attention_layer2)
    self.r_fc3 = nn.Linear(model_attention_layer2, (y_dim+z_dim))

  def forward(self,y,z,m,mode):
    size1 = np.shape(y)[0]
    size2 = np.shape(z)[0]
    size3 = np.shape(m)[0]

    if size1 != size2:
      x = tf.concat([y,z],axis=0)
    else:
      x = tf.concat([y,z],axis=1)

    if mode == 1:
      x = x.numpy()
      x = torch.Tensor(x)
      x = self.fc1(x)
      x = F.relu(x)
      x = self.fc2(x)
      x = F.relu(x)
      x = self.fc3(x)
      x = F.relu(x)
      x = self.fc4(x)
      x = F.relu(x)
      x = self.fc5(x)
      x = F.relu(x)
      x = self.fc6(x)
      x = F.sigmoid(x)

      return x

    if mode == 2:

      for param in self.fc1.parameters():
          param.requires_grad = False
      for param in self.fc2.parameters():
          param.requires_grad = False
      for param in self.fc3.parameters():
          param.requires_grad = False
      for param in self.fc4.parameters():
          param.requires_grad = False
      for param in self.fc5.parameters():
          param.requires_grad = False
      for param in self.fc6.parameters():
          param.requires_grad = False

      r = self.r_fc1(m)
      r = F.relu(r)
      r = self.r_fc2(r)
      r = F.relu(r)
      r = self.r_fc3(r)
      r = F.sigmoid(r)

      x = x.numpy()
      x = torch.Tensor(x)

      x = r*x

      x = self.fc1(x)
      x = F.relu(x)
      x = self.fc2(x)
      x = F.relu(x)
      x = self.fc3(x)
      x = F.relu(x)
      x = self.fc4(x)
      x = F.relu(x)
      x = self.fc5(x)
      x = F.relu(x)
      x = self.fc6(x)
      x = F.sigmoid(x)

      return x


def train_Model_Attention(model,train_data,train_labels,val_data,val_labels,
                          train_data_class,val_data_class,train_data_weigh,val_data_weigh,learning_rate,batch_size,epochs, path1, patience=20, factor=0.2):

  flag_start_again = 0
  while(flag_start_again != 2):
    flag_start_again = 0
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.BCELoss()

    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=patience, factor=factor, verbose=True)

    losses, epoch_lst, train_accs, val_accs  = [], [] ,[], []
    length_train = np.shape(train_data)[0]
    length_val = np.shape(val_data)[0]

    best_val_acc = 0
    tj = 0
    mode = 1
    flag_mode_2 = 0
    flag_mode_3 = 0
    for j in range(epochs):

        if flag_start_again == 1:
          break

        if mode == 2 and flag_mode_2 == 0:
          flag_mode_2 = 1
          print("Stage 2 starts now")
          optimizer = optim.Adam(model.parameters(), lr=learning_rate)
          scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=patience, factor=factor, verbose=True)

        if mode == 3:
          break

        perm = np.random.permutation(length_train)
        train_data = train_data[perm]
        train_labels = train_labels[perm]
        train_data_class = train_data_class[perm]
        train_data_weigh = train_data_weigh[perm]

        for i in range(0, length_train, batch_size):
            if (i + batch_size) > np.shape(train_data)[0]:
                break
            x_in = train_data[i:i+batch_size,:]
            y_in = train_data_class[i:i+batch_size,:]
            z_in = train_data_weigh[i:i+batch_size,:]
            x_in = torch.Tensor(x_in)
            y_in = torch.Tensor(y_in)
            z_in = torch.Tensor(z_in)
            labels_in = train_labels[i:i+batch_size]
            labels_in = torch.Tensor(labels_in).float().unsqueeze(1)

            output = model(x_in,y_in,z_in,mode)
            loss = criterion(output, labels_in)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if i % int(batch_size*300) == 0:
              losses.append(float(loss)/batch_size)  # compute *average* loss
              epoch_lst.append(tj)
              train_cost = float(loss.detach().numpy())
              train_acc = estimate_accuracy_Model_Attention(model, train_data, train_labels,train_data_class,train_data_weigh,curr_mode=mode)
              train_accs.append(train_acc)
              val_acc = estimate_accuracy_Model_Attention(model, val_data, val_labels,val_data_class,val_data_weigh,curr_mode=mode)
              val_accs.append(val_acc)
              print("Epoch %d. [Val Acc %.2f%%] [Train Acc %.2f%%, Loss %f]" % (
                    j, val_acc, train_acc, train_cost))
              tj += 1

              scheduler.step(val_acc)
              for param_group in optimizer.param_groups:
                  curr_lr = param_group['lr']
              if curr_lr == learning_rate*np.power(factor,3) and mode == 1:
                  print("Stage 1 ended. No improvement in validation accuracy for %d epochs." % patience)
                  flag_start_again = 2
                  mode = 2
                  torch.save(model, path1)
                  break

              if curr_lr == learning_rate*np.power(factor,3) and mode == 2:
                  print("Stage 2 ended. No improvement in validation accuracy for %d epochs." % patience)
                  mode = 3
                  break


              if tj == 10 and val_acc < 53:
                  flag_start_again = 1

  del y_in
  del labels_in
  return losses, epoch_lst, train_accs, val_accs


def create_labels_Model_Attention(model,data,data_class,data_weigh,params):

  data = (data - params[0])/params[1]
  data_class = (data_class - params[2])/params[3]
  data_weigh = (data_weigh - params[4])/params[5]

  data = torch.Tensor(data)
  data_class = torch.Tensor(data_class)
  data_weigh = torch.Tensor(data_weigh)
  nn_values = model(data,data_class,data_weigh,2)
  labels_out = torch.round(nn_values)

  return np.squeeze(labels_out.detach().numpy()), np.squeeze(nn_values.detach().numpy())

def create_labels_Model_Concatenate(model,data,data_class,data_weigh,params):

  data = (data - params[0])/params[1]
  data_class = (data_class - params[2])/params[3]
  data_weigh = (data_weigh - params[4])/params[5]

  data = torch.Tensor(data)
  data_class = torch.Tensor(data_class)
  data_weigh = torch.Tensor(data_weigh)
  nn_values = model(data,data_class,data_weigh,1)
  labels_out = torch.round(nn_values)

  return np.squeeze(labels_out.detach().numpy()), np.squeeze(nn_values.detach().numpy())


def estimate_accuracy_Model_Attention(model, data, labels, data_class, data_weigh, curr_mode = 2):

    data = torch.Tensor(data)
    data_class = torch.Tensor(data_class)
    data_weigh = torch.Tensor(data_weigh)
    labels = torch.Tensor(labels).float().unsqueeze(1)
    out = model(data,data_class,data_weigh,curr_mode)
    correct_mask = torch.round(out) == labels
    correct = correct_mask.sum().item()
    all = correct_mask.numel()

    return 100 * correct / all


def run_Model_Attention(labels_train,data_train,labels_val,data_val,labels_test,data_test,
                              class_feat_train,class_feat_val,class_feat_test,weigh_feat_train,weigh_feat_val,weigh_feat_test,
                              path1, path2, path3, learning_rate=1e-3, batch_size=50, epochs=100):

  model = Model_Attention()

  mu_dat, sigma_dat = find_normal_parameters(data_train)
  norm_train,norm_val,norm_test = normalize_datasets(data_train,data_val,data_test,mu_dat,sigma_dat)

  mu_class, sigma_class = find_normal_parameters(class_feat_train)
  norm_train_class,norm_val_class,norm_test_class = normalize_datasets(class_feat_train,class_feat_val,class_feat_test,mu_class,sigma_class)

  mu_weigh, sigma_weigh = find_normal_parameters(weigh_feat_train)
  norm_train_weigh,norm_val_weigh,norm_test_weigh = normalize_datasets(weigh_feat_train,weigh_feat_val,weigh_feat_test,mu_weigh,sigma_weigh)

  learning_curve_info = train_Model_Attention(model,norm_train
                                                   , labels_train, norm_val
                                                   , labels_val,norm_train_class,norm_val_class,norm_train_weigh,norm_val_weigh
                                                   , learning_rate, batch_size, epochs, path1)

  plot_learning_curve(*learning_curve_info)

  test_accuracy = estimate_accuracy_Model_Attention(model, norm_test, labels_test, norm_test_class, norm_test_weigh,2)
  print("[Test Acc %.2f%%]" % (test_accuracy))

  torch.save(model, path2)
  np.save(path3,np.array([mu_dat,sigma_dat,mu_class,sigma_class,mu_weigh,sigma_weigh,WL_classifier,learning_rate,batch_size,epochs]))

  return

#BM FCNN

In [ ]:
class Model_BM_FCNN(nn.Module):
  def __init__(self,dim=WL_classifier):
    super().__init__()
    self.fct0 = nn.Linear(dim,model_classifier_layer1)
    self.fct = nn.Linear(model_classifier_layer1,model_classifier_layer2)
    self.fc1 = nn.Linear(model_classifier_layer2,model_classifier_layer3)
    self.fc2 = nn.Linear(model_classifier_layer3,model_classifier_layer4)
    self.fc3 = nn.Linear(model_classifier_layer4,6)

  def forward(self,x):
    x = self.fct0(x)
    x = F.relu(x)
    x = self.fct(x)
    x = F.relu(x)
    x = self.fc1(x)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    x = self.fc3(x)

    return x


def train_Model_BM_FCNN(model, train_data, train_labels, val_data, val_labels, learning_rate, batch_size, epochs, patience=20, factor=0.2):

    flag_start_again = 0
    while(flag_start_again != 2):
      flag_start_again = 0

      optimizer = optim.Adam(model.parameters(), lr=learning_rate)
      criterion = nn.CrossEntropyLoss()
      # Learning rate scheduler: ReduceLROnPlateau
      scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=patience, factor=factor, verbose=True)

      losses, epoch_lst, train_accs, val_accs = [], [], [], []
      length_train = np.shape(train_data)[0]
      length_val = np.shape(val_data)[0]

      best_val_acc = 0
      tj = 0
      flagy = 0
      for j in range(epochs):
          if flagy == 1 or flag_start_again == 1:
            break
          perm = np.random.permutation(length_train)
          train_data = train_data[perm]
          train_labels = train_labels[perm]

          for i in range(0, length_train, batch_size):
              if (i + batch_size) > length_train:
                  break
              y_in = train_data[i:i+batch_size, :]
              labels_in = train_labels[i:i+batch_size]
              y_in = torch.Tensor(y_in)
              labels_in = torch.Tensor(labels_in).long()
              output = model(y_in)
              loss = criterion(output, labels_in)
              optimizer.zero_grad()
              loss.backward()
              optimizer.step()

              if i % int(batch_size*300) == 0:
                losses.append(float(loss)/batch_size)  # compute *average* loss
                epoch_lst.append(tj)
                train_cost = float(loss.detach().numpy())
                train_acc = estimate_accuracy_Model_BM_FCNN(model, train_data, train_labels)
                train_accs.append(train_acc)
                val_acc = estimate_accuracy_Model_BM_FCNN(model, val_data, val_labels)
                val_accs.append(val_acc)
                print("Epoch %d. [Val Acc %.2f%%] [Train Acc %.2f%%, Loss %f]" % (
                      j, val_acc, train_acc, train_cost))
                tj += 1

                scheduler.step(val_acc)
                for param_group in optimizer.param_groups:
                    curr_lr = param_group['lr']
                if curr_lr == learning_rate*np.power(factor,3):
                    print("Training stopped. No improvement in validation accuracy for %d epochs." % patience)
                    flagy = 1
                    flag_start_again = 2
                    break

                if tj == 10 and val_acc < 53:
                  flag_start_again = 1

    del y_in
    del labels_in
    return losses, epoch_lst, train_accs, val_accs


def estimate_accuracy_Model_BM_FCNN(model, data, labels):

    data = torch.Tensor(data)
    labels = torch.Tensor(labels).long()
    out = model(data)
    correct_mask = torch.argmax(out,axis=1) == labels
    correct = correct_mask.sum().item()
    all = correct_mask.numel()

    return 100 * correct / all

def create_labels_Model_BM_FCNN(model,data,params):

  data = (data - params[0])/params[1]
  data = torch.Tensor(data)
  nn_values = model(data)
  labels_out = torch.round(nn_values)

  return np.squeeze(labels_out.detach().numpy()), np.squeeze(nn_values.detach().numpy())


def run_Model_BM_FCNN(labels_train,data_train,labels_val,data_val,labels_test,data_test,path1, path2, learning_rate=1e-3, batch_size=50, epochs=100):
  model = Model_BM_FCNN()

  mu, sigma = find_normal_parameters(data_train)
  norm_train,norm_val,norm_test = normalize_datasets(data_train,data_val,data_test,mu,sigma)

  learning_curve_info = train_Model_BM_FCNN(model,norm_train
                                                   , labels_train, norm_val
                                                   , labels_val, learning_rate, batch_size, epochs)

  plot_learning_curve(*learning_curve_info)

  test_accuracy = estimate_accuracy_Model_BM_FCNN(model, norm_test, labels_test)
  print("[Test Acc %.2f%%]" % (test_accuracy))

  torch.save(model, path1)
  np.save(path2,np.array([mu,sigma,WL_classifier,learning_rate,batch_size,epochs]))

  return

#BM FCNN12

In [ ]:
class Model_BM_FCNN1(nn.Module):
  def __init__(self,dim=WL_classifier):
    super().__init__()
    self.fct0 = nn.Linear(dim,model_classifier_layer1)
    self.fct = nn.Linear(model_classifier_layer1,model_classifier_layer2)
    self.fc1 = nn.Linear(model_classifier_layer2,model_classifier_layer3)
    self.fc2 = nn.Linear(model_classifier_layer3,model_classifier_layer4)
    self.fc3 = nn.Linear(model_classifier_layer4,model_classifier_layer5)
    self.fc4 = nn.Linear(model_classifier_layer5,1)

  def forward(self,x):
    x = self.fct0(x)
    x = F.relu(x)
    x = self.fct(x)
    x = F.relu(x)
    x = self.fc1(x)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    x = self.fc3(x)
    x = F.relu(x)
    x = self.fc4(x)
    x = F.sigmoid(x)

    return x


def train_Model_BM_FCNN1(model, train_data, train_labels, val_data, val_labels, learning_rate, batch_size, epochs, patience=20, factor=0.2):

    flag_start_again = 0
    while(flag_start_again != 2):
      flag_start_again = 0

      optimizer = optim.Adam(model.parameters(), lr=learning_rate)
      criterion = nn.BCELoss()

      # Learning rate scheduler: ReduceLROnPlateau
      scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=patience, factor=factor, verbose=True)

      losses, epoch_lst, train_accs, val_accs = [], [], [], []
      length_train = np.shape(train_data)[0]
      length_val = np.shape(val_data)[0]

      best_val_acc = 0
      tj = 0
      flagy = 0
      for j in range(epochs):
          if flagy == 1 or flag_start_again == 1:
            break
          perm = np.random.permutation(length_train)
          train_data = train_data[perm]
          train_labels = train_labels[perm]

          for i in range(0, length_train, batch_size):
              if (i + batch_size) > length_train:
                  break
              y_in = train_data[i:i+batch_size, :]
              labels_in = train_labels[i:i+batch_size]
              y_in = torch.Tensor(y_in)
              labels_in = torch.Tensor(labels_in).float().unsqueeze(1)

              output = model(y_in)
              loss = criterion(output, labels_in)
              optimizer.zero_grad()
              loss.backward()
              optimizer.step()

              if i % int(batch_size*300) == 0:
                losses.append(float(loss)/batch_size)  # compute *average* loss
                epoch_lst.append(tj)
                train_cost = float(loss.detach().numpy())
                train_acc = estimate_accuracy_Model_BM_FCNN1(model, train_data, train_labels)
                train_accs.append(train_acc)
                val_acc = estimate_accuracy_Model_BM_FCNN1(model, val_data, val_labels)
                val_accs.append(val_acc)
                print("Epoch %d. [Val Acc %.2f%%] [Train Acc %.2f%%, Loss %f]" % (
                      j, val_acc, train_acc, train_cost))
                tj += 1

                scheduler.step(val_acc)
                for param_group in optimizer.param_groups:
                    curr_lr = param_group['lr']
                if curr_lr == learning_rate*np.power(factor,3):
                    print("Training stopped. No improvement in validation accuracy for %d epochs." % patience)
                    flagy = 1
                    flag_start_again = 2
                    break

                if tj == 10 and val_acc < 53:
                  flag_start_again = 1

    del y_in
    del labels_in
    return losses, epoch_lst, train_accs, val_accs

def estimate_accuracy_Model_BM_FCNN1(model, data, labels):

    data = torch.Tensor(data)
    labels = torch.Tensor(labels).float().unsqueeze(1)
    out = model(data)
    correct_mask = torch.round(out) == labels
    correct = correct_mask.sum().item()
    all = correct_mask.numel()

    return 100 * correct / all

def create_labels_Model_BM_FCNN1(model,data,params):

  data = (data - params[0])/params[1]
  data = torch.Tensor(data)
  nn_values = model(data)
  labels_out = torch.round(nn_values)

  return np.squeeze(labels_out.detach().numpy()), np.squeeze(nn_values.detach().numpy())


def run_Model_BM_FCNN1(labels_train,data_train,labels_val,data_val,labels_test,data_test,path1, path2, learning_rate=1e-3, batch_size=50, epochs=100):
  model = Model_BM_FCNN1()

  mu, sigma = find_normal_parameters(data_train)
  norm_train,norm_val,norm_test = normalize_datasets(data_train,data_val,data_test,mu,sigma)

  learning_curve_info = train_Model_BM_FCNN1(model,norm_train
                                                   , labels_train, norm_val
                                                   , labels_val, learning_rate, batch_size, epochs)

  plot_learning_curve(*learning_curve_info)

  test_accuracy = estimate_accuracy_Model_BM_FCNN1(model, norm_test, labels_test)
  print("[Test Acc %.2f%%]" % (test_accuracy))

  torch.save(model, path1)
  np.save(path2,np.array([mu,sigma,WL_classifier,learning_rate,batch_size,epochs]))

  return